Here, the performance of the lookup tables optimized in the "example_F_lookup_training_test.ipynb" on state stabilization for longer time durations can be tested.

In [4]:
# ruff: noqa
import sys, os
sys.path.append(os.path.abspath("./../feedback-grape"))
sys.path.append(os.path.abspath("./../"))

# ruff: noqa
from helpers import (
    transport_unitary,
    initialize_chain_of_zeros,
    partial_trace,
    generate_all_states,
    test_implementations,
)
import numpy as np
from feedback_grape.utils.fidelity import fidelity
from feedback_grape.utils.operators import identity, sigmam, sigmap
from library.utils.qubit_chain_1D import embed
from feedback_grape.utils.solver import mesolve
from tqdm import tqdm

test_implementations()

In [5]:
# Physical parameters
n = 2 # number of qubits per chain (>= 3)
N_chains = 2 # Number of parallel chains to simulate
gamma = 0.25 # Decay constant
evaluation_time_steps = 200 # Number of time steps for evaluation

# Path to save results
results_path = "./evaluation_results/baseline_fidelities.npz"

In [6]:
# Initialization
base_dim = 2**N_chains
T_half = transport_unitary(0.5, n, N_chains)
c_ops = [sum([gamma * embed(sigmam(), idx, N_chains*n) for idx in range(N_chains*n)])]

# Evolve all basis states and compute fidelities
states = [np.array(state) for state in generate_all_states(N_chains)]
initial_states = [np.array(state) for state in states]
fidelities_each = np.zeros((len(states), evaluation_time_steps+1))
fidelities_each[:, 0] = 1.0

for i in tqdm(range(evaluation_time_steps)):
    for j, rho in enumerate(states):
        assert np.all(np.isclose(rho, rho.conj().T)), "State is not Hermitian"
        assert np.isclose(np.trace(rho).real, 1.0), "State is not normalized"
        assert np.all(np.linalg.eigvalsh(rho) >= -1e-10), "State is not positive semidefinite"

        # Propagate the state
        tmp = initialize_chain_of_zeros(rho, n=n, N_chains=N_chains)
        tmp = T_half @ tmp @ T_half.conj().T

        tmp = mesolve(
            jump_ops=c_ops,
            rho0=tmp,
        )

        tmp = T_half @ tmp @ T_half.conj().T
        tmp = partial_trace(tmp, sys_A_dim=base_dim**(n-1), sys_B_dim=base_dim)

        states[j] = tmp
        fid = fidelity(C_target=initial_states[j], U_final=tmp, evo_type="density")
        fidelities_each[j, i+1] = fid

fidelities_each = fidelities_each.mean(axis=0) # Average over all basis states

# Reshape back to matrix form
np.savez(results_path, fidelities=fidelities_each)

100%|██████████| 200/200 [01:45<00:00,  1.89it/s]
